In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load in 

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the "../input/" directory.
# For example, running this (by clicking run or pressing Shift+Enter) will list the files in the input directory

import os
print(os.listdir("../input"))

# Any results you write to the current directory are saved as output.

In [ ]:
from sklearn.feature_extraction.text import TfidfVectorizer

# data preprocessing
train_data = pd.read_json('../input/train.json')
test_data = pd.read_json('../input/test.json')

train_data['ingredients'] = train_data['ingredients'].apply(lambda list: ','.join(list).lower())
test_data['ingredients'] = test_data['ingredients'].apply(lambda list: ','.join(list).lower())

vectorizer = TfidfVectorizer(binary = True)
train_X = vectorizer.fit_transform(train_data['ingredients'])
test_X = vectorizer.transform(test_data['ingredients'])

In [ ]:
idxtuple = train_X.nonzero()
for i in range(16):
    row = idxtuple[0][i]
    col = idxtuple[1][i]
    print('Recipe {0}: {1} = {2}'.format(row, vectorizer.get_feature_names()[col], train_X[row, col]))

In [ ]:
# Naive Bayes, score = 0.68634
'''
from sklearn.naive_bayes import MultinomialNB

clf = MultinomialNB().fit(train_X, train_data['cuisine'])

test_pred = clf.predict(test_X)
print(test_pred)
answer = pd.DataFrame({'id': test_data['id'], 'cuisine': test_pred})
answer.to_csv('answer.csv', index = False)
'''

In [ ]:
# SVC with panelty = 100, socre = 0.71721, but take too long time
'''
from sklearn.svm import SVC

clf = SVC(C = 100).fit(train_X, train_data['cuisine'])

test_pred = clf.predict(test_X)
print(test_pred)
answer = pd.DataFrame({'id': test_data['id'], 'cuisine': test_pred})
answer.to_csv('answer.csv', index = False)
'''

In [ ]:
# LinearSVC with OvR classifier, score = 0.78761 and very fast
'''
from sklearn.multiclass import OneVsRestClassifier
from sklearn.svm import LinearSVC
clf = OneVsRestClassifier(LinearSVC(random_state=0)).fit(train_X, train_data['cuisine'])

test_pred = clf.predict(test_X)
print(test_pred)
answer = pd.DataFrame({'id': test_data['id'], 'cuisine': test_pred})
answer.to_csv('answer.csv', index = False)
'''

In [ ]:
# LinearSVC with OvO classifier, score = 0.78821
'''
from sklearn.multiclass import OneVsOneClassifier
from sklearn.svm import LinearSVC
clf = OneVsOneClassifier(LinearSVC(random_state=0)).fit(train_X, train_data['cuisine'])

test_pred = clf.predict(test_X)
print(test_pred)
answer = pd.DataFrame({'id': test_data['id'], 'cuisine': test_pred})
answer.to_csv('answer.csv', index = False)
'''

以下的code是根據我的猜測實作的，看起來不像machine learning，而是單純透過統計學和邏輯推論組成的答案。我的想法如下：

* 首先製作一個二維矩陣，column是不同的料理種類，row是不同的食材，以此矩陣統計各類食材在不同的料理當中被使用的頻率
* 欲得出一種食材在某料理種類當中的特殊程度，可以透過分析該食材在不同料理當中出現的次數，如果在某料理當中出現該食材的次數特別多，表示這個食材很可能是該料理中的特色食材
* 但是有些食材無論在哪種料理當中都很常出現，例如鹽巴(salt)、糖(sugar)、油(oil)等，這些食材對於一道菜是屬於什麼料理是幾乎沒有影響的，所以其實他們不是那麼重要
* 相對的，如果資料集當中某種料理的數量特別多，那麼任何一種食材出現在該料理當中的機率都會相對於出現在其它料理當中的機率還要高，所以在比較同一樣食材在不同料理當中出現的頻率之前，必須先針對每一種料理下不同食材的出現次數做正規化（將次數轉換為百分比），這樣才可以在公平的基準點下比較
* 在做完正規化之後，就可以比較某一樣食材在不同的料理間使用的狀況。這裡我希望將一樣食材用一個單位向量表示，這個向量的長度與料理的種類數量一樣（20維，因為有20種料理），每一個維度的值代表「此食材有多麼的OO料理」。比如味噌在日本料理當中大約有10%的比例被使用，但是在其它料理當中都是0%，所以這就是一個平行日本料理那個方向的單位向量，表示這個食材很日本料理。又比如鹽巴無論在那一種料理當中，被使用的頻率大約都是50%，那鹽巴就是一個指向各個維度正向的單位向量，無論在哪個維度上的投影都差不多長，也就是他沒有特別代表什麼料理。

-> 這其實就是Tfidf的概念（。

* 在產生出所有食材的向量之後，每一個食譜都可以視為是一群向量的疊加，所以我就把一個食譜當中有用到的食材的向量全部加總，得到一個很大的向量，用這個向量代表這個食譜。最後再尋找這個食譜在哪個維度的值是最大的，則這個食譜很有可能就是那個維度所對應的料理種類
* 如果在testing data上遭遇了沒有見過的食材，則先假設這個食材的向量為0向量，即不考慮這個沒見過的東西對料理種類的影響，先以既有的知識去猜測。在predict完第一輪之後就會有一個粗略的結果，再根據這個結果去對沒有見過的食材做統計，並且分析他是屬於哪一種料理，再用這個結果去predict第二次。有點類似semi-supervised learning的感覺。


In [ ]:
# My toy model(?)
train_data = pd.read_json('../input/train.json')
test_data = pd.read_json('../input/test.json')

cuisine = train_data['cuisine'].value_counts()
ingredients = {}
for idx, row in train_data.iterrows():
    for i in range(len(row['ingredients'])):
        ingr = row['ingredients'][i].lower()
        if row['cuisine'] in ingredients:
            if ingr in ingredients[row['cuisine']]:
                ingredients[row['cuisine']][ingr] += 1
            else:
                ingredients[row['cuisine']][ingr] = 1
        else:
            ingredients[row['cuisine']] = {}
            ingredients[row['cuisine']][ingr] = 1

ingredients = pd.DataFrame(ingredients)
ingredients = ingredients.fillna(0)

In [ ]:
# term frequency
ingredients = ingredients.apply(lambda x: x.apply(lambda y: y / cuisine[x.name]))

# unit-vectorize
def unit_vec(vec):
    len = (vec.apply(lambda x: x * x).sum()) ** 0.5
    return vec.apply(lambda x: x / len)
ingredients = ingredients.apply(unit_vec, axis = 1)

In [ ]:
# get recipe vector on training data
train_data_vector = []
for idx, row in train_data.iterrows():
    train_data_vector.append({
        'id': row['id'],
        'cuisine': row['cuisine'],
        'cuisine_vector': ingredients.loc[row['ingredients']].sum()
    })

# Evaluate
total = len(train_data_vector)
correct = 0
for i in range(total):
    if train_data_vector[i]['cuisine_vector'].idxmax() == train_data_vector[i]['cuisine']:
        correct += 1

print('Result on training data')
print(f'Total: {total}, Correct: {correct}, Accuracy: {correct / total}')

In [ ]:
# get recipe vector on testing data and predict(?)
testing_data_vector = []
for idx, row in test_data.iterrows():
    ingredients_known = [igdt for igdt in row['ingredients'] if igdt in ingredients.index]
    vector = ingredients.loc[ingredients_known].sum()
    cuisine = vector.idxmax()
    testing_data_vector.append({
        'id': row['id'],
        'cuisine_vector': vector,
        'cuisine': cuisine
    })
answer = pd.DataFrame(testing_data_vector)[['id','cuisine']]
answer.to_csv('answer.csv', index = False)